# Konvertering av Bergens registeret, Person sammenkoblinger



In [1]:
from hentOgVaskData import hent_og_vask_data
import polars as pl
pl.Config.set_tbl_rows(30)

polars.config.Config

In [3]:
'''
1. Leser inn Data exportert i fra Bergens Access database.
   I fra den tabellen som heter `tblInnmeldte`. Dette gjøres ved å benytte funksjonen `hent_og_vask_data()`. Denne funksjonen er også hardkodet med de som har dobbelt medlemskap mellom registerene. Disse blir satt til "venn" i stedet for medlem av Bergens registeret.
'''

df = hent_og_vask_data()
# Medlems sjekk, det er viktig at dette tallet er nokså likt som rapportert in til statsforvalteren 282
print("Antall medlemmer i Bergens registeret -->", df.filter(pl.col("Kategori").str.starts_with("M")).shape)
print("Alle data i fra excel fil -->", df.shape)

Could not determine dtype for column 43, falling back to string


Antall medlemmer i Bergens registeret --> (279, 46)
Alle data i fra excel fil --> (577, 46)


In [3]:
'''
2. Henter ut alle personer som har andre personer knyttet til seg.
   Det lages en rad for hver kobling en person har. Mulige koblinger er "SamID", "MorID", "FarID"
   Fornavn og etternavn hentes også på personen som har tilknyttninger
'''

df_connectedMedlemer = pl.concat([
    df.filter(pl.col(c).is_not_null())
      .select(
          pl.col("FORNAVN"),
          pl.col("EFTERNAVN"),
          pl.lit(c).alias("type"),
          pl.col("MedlemID"),
          pl.col(c).alias("ConnectedMedlemID")
      )
    for c in ["SamID", "MorID", "FarID"]
])
print("Personer som er knyttet til andre personer mor, far, sambo -->", df_connectedMedlemer)

Personer som er knyttet til andre personer mor, far, sambo --> shape: (436, 5)
┌────────────────────────────────┬────────────────────┬───────┬──────────┬───────────────────┐
│ FORNAVN                        ┆ EFTERNAVN          ┆ type  ┆ MedlemID ┆ ConnectedMedlemID │
│ ---                            ┆ ---                ┆ ---   ┆ ---      ┆ ---               │
│ str                            ┆ str                ┆ str   ┆ i64      ┆ i64               │
╞════════════════════════════════╪════════════════════╪═══════╪══════════╪═══════════════════╡
│ Martin Daniel Peter            ┆ Aeschlimann        ┆ SamID ┆ 13812    ┆ 13617             │
│ Terje                          ┆ Alræk              ┆ SamID ┆ 13803    ┆ 12705             │
│ Gry Veronica                   ┆ Alsos              ┆ SamID ┆ 14800    ┆ 12729             │
│ Vahan                          ┆ Babayan            ┆ SamID ┆ 14792    ┆ 14631             │
│ Jorun                          ┆ Barane             ┆ SamID ┆ 15

In [4]:
'''
3. Finner alle MedlemID'er på sammenknyttede personer som ikke eksisterer.
   Sletter så alle disse fra vår liste i fra steg 2.
'''


# 1. Anti-join: IDs in connected that aren't in df
missing_ids = df_connectedMedlemer.join(
    df, 
    left_on="ConnectedMedlemID",      # Key in the left DataFrame
    right_on="MedlemID",              # Key in the right DataFrame
    how="anti"
).select(pl.col("ConnectedMedlemID").unique().alias("MedlemID"))
print("missing ids -->", missing_ids)

# Sletter alle ID'er som ikke finnes fra vår Connected liste
# Sletter alle koblinger til personer som ikke finnes i registeret
df_connectedMedlemer = df_connectedMedlemer.join(
    missing_ids,
    left_on="ConnectedMedlemID",      # Key in the left DataFrame
    right_on="MedlemID",              # Key in the right DataFrame
    how="anti"
)
print("Liste over samenkoblede personer, etter at ugyldige er slettet -->", df_connectedMedlemer)

missing ids --> shape: (35, 1)
┌──────────┐
│ MedlemID │
│ ---      │
│ i64      │
╞══════════╡
│ 11025    │
│ 11112    │
│ 11113    │
│ 11212    │
│ 11215    │
│ 11243    │
│ 11595    │
│ 11596    │
│ 11627    │
│ 11677    │
│ 12121    │
│ 12507    │
│ 12958    │
│ 13134    │
│ 13170    │
│ …        │
│ 14050    │
│ 14325    │
│ 14327    │
│ 14331    │
│ 14361    │
│ 14363    │
│ 14447    │
│ 14470    │
│ 14551    │
│ 14652    │
│ 14743    │
│ 15444    │
│ 15445    │
│ 15485    │
│ 15533    │
└──────────┘
Liste over samenkoblede personer, etter at ugyldige er slettet --> shape: (380, 5)
┌────────────────────────────────┬────────────────────┬───────┬──────────┬───────────────────┐
│ FORNAVN                        ┆ EFTERNAVN          ┆ type  ┆ MedlemID ┆ ConnectedMedlemID │
│ ---                            ┆ ---                ┆ ---   ┆ ---      ┆ ---               │
│ str                            ┆ str                ┆ str   ┆ i64      ┆ i64               │
╞════════════════════════

In [6]:
'''
4. Henter inn fornavn og etternavn til den personen som er tilknyttet
   Denne listen benyttes til å koble alle personer i Cornerstone etter at alle personer er lagt inn.
   Cornerstone krever å ha med fornavn og etternavn i tillegg til "MedlmID" som identifikator
'''
# Perform the join by preparing the right side first
df_connectedMedlemer = df_connectedMedlemer.join(
    # 1. Select and rename columns from 'df' on the fly
    df.select(
        pl.col("MedlemID"),  # The join key must be included
        pl.col("FORNAVN").alias("Connected_FORNAVN"),
        pl.col("EFTERNAVN").alias("Connected_EFTERNAVN")
    ),
    left_on="ConnectedMedlemID",
    right_on="MedlemID",
    how="left"  # Using 'left' is safer to not lose rows from the left table
)
print("Liste over samenkoblede personer, brukes til konvertering av relasjoner -->", df_connectedMedlemer)

Liste over samenkoblede personer, brukes til konvertering av relasjoner --> shape: (380, 7)
┌───────────────┬───────────────┬───────┬──────────┬───────────────┬───────────────┬───────────────┐
│ FORNAVN       ┆ EFTERNAVN     ┆ type  ┆ MedlemID ┆ ConnectedMedl ┆ Connected_FOR ┆ Connected_EFT │
│ ---           ┆ ---           ┆ ---   ┆ ---      ┆ emID          ┆ NAVN          ┆ ERNAVN        │
│ str           ┆ str           ┆ str   ┆ i64      ┆ ---           ┆ ---           ┆ ---           │
│               ┆               ┆       ┆          ┆ i64           ┆ str           ┆ str           │
╞═══════════════╪═══════════════╪═══════╪══════════╪═══════════════╪═══════════════╪═══════════════╡
│ Martin Daniel ┆ Aeschlimann   ┆ SamID ┆ 13812    ┆ 13617         ┆ Drude         ┆ Isene         │
│ Peter         ┆               ┆       ┆          ┆               ┆               ┆               │
│ Terje         ┆ Alræk         ┆ SamID ┆ 13803    ┆ 12705         ┆ Torild        ┆ Alræk         │